In [57]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
from statsmodels.graphics.factorplots import interaction_plot
import matplotlib.pyplot as plt
from scipy import stats

In [58]:
datafile = "ToothGrowth.csv"
data = pd.read_csv(datafile)

### Calculation of Sum of Squares
Perhitungan **jumlah kuadrat (sum of squares)** atau **variasi dalam data** sebenarnya cukup sederhana jika menggunakan Python. Pertama, kita mulai dengan mendapatkan **ukuran sampel (N)** dan **derajat kebebasan (degree of freedom)** yang diperlukan. Nilai-nilai ini akan kita gunakan nanti untuk menghitung **mean square** (rata-rata kuadrat). Setelah kita memiliki derajat kebebasan, kita dapat melanjutkan dengan perhitungan **jumlah kuadrat (sum of square)**.

### Degree of Freedom

In [59]:
N = len(data.len)
df_a = len(data.supp.unique()) - 1
df_b = len(data.dose.unique()) - 1
df_axb = df_a*df_b 
df_w = N - (len(data.supp.unique())*len(data.dose.unique()))

print(f"N = {N}")
print(f"df_a = {df_a}")
print(f"df_b = {df_b}")
print(f"df_axb = {df_axb}")
print(f"df_w = {df_w}")

N = 60
df_a = 1
df_b = 2
df_axb = 2
df_w = 54


### Sum of Squares
Untuk perhitungan **jumlah kuadrat A, B, dan Total**, kita perlu memiliki **grand mean** (rata-rata keseluruhan). Dengan menggunakan metode `mean` dari **Pandas DataFrame** pada **variabel dependen saja**, kita akan mendapatkan **grand mean** tersebut.

In [60]:
grand_mean = data['len'].mean()
print(f"grand mean = {grand_mean}")

grand mean = 18.813333333333336


### Sum of Squares A – supp
Menghitung Sum of Squares dari faktor A (supp).

In [61]:
ssq_a = sum([(data[data.supp ==l].len.mean()-grand_mean)**2 for l in data.supp])
print(f"ssq_a = {ssq_a}")

ssq_a = 205.35000000000008


### Sum of Squares B – dose
Menghitung Sum of Squares dari faktor B (dose).

In [62]:
ssq_b = sum([(data[data.dose ==l].len.mean()-grand_mean)**2 for l in data.dose])
print(f"ssq_b = {ssq_b}")

ssq_b = 2426.4343333333336


### Sum of Squares Total

In [63]:
ssq_t = sum((data.len - grand_mean)**2)
print(f"ssq_total = {ssq_t}")

ssq_total = 3452.2093333333332


### Sum of Squares Within (error/residual)
Selanjutnya, kita perlu menghitung Sum of Squares Within yang terkadang disebut juga sebagai error atau residual.

In [64]:
vc = data[data.supp == 'VC']
oj = data[data.supp == 'OJ']
vc_dose_means = [vc[vc.dose == d].len.mean() for d in vc.dose]
oj_dose_means = [oj[oj.dose == d].len.mean() for d in oj.dose]

In [65]:
ssq_w = sum(((oj.len - oj_dose_means)**2) + sum((vc.len - vc_dose_means)**2))

### Sum of Squares interaction
Karena kita menggunakan two-way, maka kita perlu menghitung Sum of Square dari interaksi A dan B

In [66]:
ssq_axb = ssq_t-ssq_a-ssq_b-ssq_w

### Mean Squares
Kita kemudian melanjutkkan dengan menghitung Mean Square dari masing masing faktor, interaksi antar faktor, dan error/residu (within).

In [67]:
# Mean Square A
ms_a = ssq_a/df_a

# Mean Square B
ms_b = ssq_b/df_b

# Mean Square AxB
ms_axb = ssq_axb/df_axb

# Mean Square Within/Error/Residual
ms_w = ssq_w/df_w

### F-ratio
Nilai **F-statistic** diperoleh dengan membagi **mean square** dari masing-masing efek dan interaksi dengan **mean square dalam kelompok (within)**, yang juga disebut sebagai **error** atau **residual**.


In [68]:
# F-ratio A
f_a = ms_a/ms_w
# F-ratio B
f_b = ms_b/ms_w
# F-ratio AxB
f_axb = ms_axb/ms_w

### p-value
Kita dapat menggunakan metode `f.sf` dari **scipy.stats** untuk memeriksa apakah **nilai F** yang kita peroleh berada di atas **nilai kritis**. Untuk melakukannya, kita perlu menggunakan **nilai F** dari setiap efek dan interaksi, serta **derajat kebebasan (degree of freedom)** untuk masing-masing efek tersebut, dan juga derajat kebebasan **dalam kelompok (within)**.


In [69]:
p_a = stats.f.sf(f_a, df_a, df_w)
p_b = stats.f.sf(f_b, df_b, df_w)
p_axb = stats.f.sf(f_axb, df_axb, df_w)

Saat ini, hasil perhitungan disimpan dalam banyak variabel. Untuk mendapatkan hasil yang **lebih mudah dibaca**, kita bisa membuat sebuah **DataFrame** yang akan memuat **tabel ANOVA** kita.

In [70]:
results = {'sum_sq':[ssq_a, ssq_b, ssq_axb, ssq_w],
           'df':[df_a, df_b, df_axb, df_w],
           'F':[f_a, f_b, f_axb, 'NaN'],
            'PR(&gt;F)':[p_a, p_b, p_axb, 'NaN']}
columns=['sum_sq', 'df', 'F', 'PR(&gt;F)']
aov_table1 = pd.DataFrame(results, columns=columns,
                          index=['supp', 'dose', 
                          'supp:dose', 'Residual'])

In [71]:
def eta_squared(aov):
    aov['eta_sq'] = 'NaN'
    aov['eta_sq'] = aov[:-1]['sum_sq']/sum(aov['sum_sq'])
    return aov
def omega_squared(aov):
    mse = aov['sum_sq'].iloc[-1] / aov['df'].iloc[-1]
    aov['omega_sq'] = 'NaN'
    aov['omega_sq'] = (aov[:-1]['sum_sq']-(aov[:-1]['df']*mse))/(sum(aov['sum_sq'])+mse)
    return aov
eta_squared(aov_table1)
omega_squared(aov_table1)
aov_table1

,sum_sq,df,F,PR(&gt;F),eta_sq,omega_sq
supp,205.350000,1,1.072413,0.305016,0.059484,0.003805
dose,2426.434333,2,6.335868,0.003374,0.702864,0.560823
supp:dose,-9519.710000,2,-24.857719,1.0,-2.757570,-2.717758
Residual,10340.135000,54,NaN,NaN,NaN,NaN


### KESIMPULAN

1. supp (Jenis Suplemen):
* F = 1.072, p = 0.305 ⟶ Tidak signifikan.
* Kesimpulan: Jenis suplemen tidak memiliki pengaruh signifikan terhadap hasil.
* Eta Squared 0.059 ⟶ Sekitar 5.9% variasi dijelaskan oleh jenis suplemen.
* Omega Squared 0.0038 ⟶ Efek sebenarnya sangat kecil, hampir nol.

2. dose (Dosis):
* F = 6.336, p = 0.00337 ⟶ Signifikan (karena p < 0.05).
* Kesimpulan: Dosis memiliki pengaruh signifikan terhadap variabel dependen.
* Eta Squared 0.70 ⟶ Sekitar 70% variasi dijelaskan oleh dosis.
* Omega Squared 0.56 ⟶ Efek sebenarnya masih besar, meskipun lebih konservatif.